Name: Kaan Aytekin

I hereby declare that I observed the honour code of the university when preparing the homework.

## Pr?gr?mm?ng?H?m?w?rk

In this exercise we model a string of text using a Markov(1) model. For simplicity we only consider letters 'a-z'. Capital letters 'A-Z' are mapped to the corresponding ones. All remaining letters, symbols, numbers, including spaces, are denoted by '.'.


We have a probability table $T$ where $T_{i,j} = p(x_t = j | x_{t-1} = i)$  transition model of letters in English text for $t=1,2 \dots N$. Assume that the initial letter in a string is always a space denoted as $x_0 = \text{'.'}$. Such a model where the probability table is always the same is sometimes called a stationary model.

1. For a given $N$, write a program to sample random strings with letters $x_1, x_2, \dots, x_N$ from $p(x_{1:N}|x_0)$
1. Now suppose you are given strings with missing letters, where each missing letter is denoted by a question mark (or underscore, as below). Implement a method, that samples missing letters conditioned on observed ones, i.e., samples from $p(x_{-\alpha}|x_{\alpha})$ where $\alpha$ denotes indices of observed letters. For example, if the input is 't??.', we have $N=4$ and
$x_1 = \text{'t'}$ and $x_4 = \text{'.'}$, $\alpha=\{1,4\}$ and $-\alpha=\{2,3\}$. Your program may possibly generate the strings 'the.', 'twi.', 'tee.', etc. Hint: make sure to make use all data given and sample from the correct distribution. Implement the method and print the results for the test strings below. 
1. Describe a method for filling in the gaps by estimating the most likely letter for each position. Hint: you need to compute
$$
x_{-\alpha}^* = \arg\max_{x_{-\alpha}} p(x_{-\alpha}|x_{\alpha})
$$
Implement the method and print the results for the following test strings along with the log-probability  $\log p(x_{-\alpha}^*,x_{\alpha})$.
1. Discuss how you can improve the model to get better estimations.

In [1]:
test_strings = ['th__br__n.f_x.', '_u_st__n_.to_be._nsw_r__','i__at_._a_h_n_._e_r_i_g','q___t.___z._____t.__.___.__.']

Hint: The code below loads a table of transition probabilities for English text.

In [2]:
import csv
from IPython.display import display, Latex

alphabet = [chr(i+ord('a')) for i in range(26)]
alphabet.append('.')
letter2idx = {c:i for i,c in enumerate(alphabet)}

T = []
with open('transitions.csv') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    for row in reader:
        T.append(row)

print('Example')
## p(x_t = 'u' | x_{t-1} = 'q')
display(Latex(r"$p(x_t = \text{'u'} | x_{t-1} = \text{'q'})$"))
print(T[letter2idx['q']][letter2idx['u']])
display(Latex(r"$p(x_t | x_{t-1} = \text{'a'})$"))
for c,p in zip(alphabet,T[letter2idx['a']]):
    print(c,p)

Example


<IPython.core.display.Latex object>

0.9949749


<IPython.core.display.Latex object>

a 0.0002835
b 0.0228302
c 0.0369041
d 0.0426290
e 0.0012216
f 0.0075739
g 0.0171385
h 0.0014659
i 0.0372661
j 0.0002353
k 0.0110124
l 0.0778259
m 0.0260757
n 0.2145354
o 0.0005459
p 0.0195213
q 0.0001749
r 0.1104770
s 0.0934290
t 0.1317960
u 0.0098029
v 0.0306574
w 0.0088799
x 0.0009562
y 0.0233701
z 0.0018701
. 0.0715219


In [7]:
import numpy as np
import copy
import itertools
TMatrix=np.matrix(T,dtype=float)
FirstLetterProbs=T[len(T)-1]
def RandomTextGenerator(N):
	word=(N+1)*['.']
	letter0='.'
	previousLetter=letter0
	for i in range(N):
		randomNumber=np.random.random()
		TransitionProbs=T[letter2idx[previousLetter]]
		cumulativeDist=np.cumsum(sorted(TransitionProbs))
		for j in range(27):
			if cumulativeDist[j]>randomNumber:
				selectedLetterIndex=j
				break
		nextLetterIndex=np.argsort(TransitionProbs)[j]
		nextLetter=list(letter2idx.keys())[nextLetterIndex]
		word[i+1]=nextLetter
		previousLetter=nextLetter
	return(''.join(word))

def TextSampler(test_strings,PredictionMultiplier):
	for text in test_strings:
		for times in range(PredictionMultiplier):	
			knownLetterIndeces=[i for i,letter in enumerate(text) if letter!='_']
			#knownLetterIndeces={i:letter for i,letter in enumerate(text) if letter!='_'}
			unknownLetterIndeces=[i for i,letter in enumerate(text) if letter=='_']
			#unknownLetterIndeces={i:letter for i,letter in enumerate(text) if letter=='_'}
			foundLetters={}
			for i,letter in enumerate(text):
				if i in unknownLetterIndeces:
					if i==0:
						firstKnownLetterIndex=knownLetterIndeces[0]
						TransitionMat=np.identity(len(T))
						for _ in range(firstKnownLetterIndex):
							TransitionMat=np.matmul(TransitionMat,TMatrix)
						LikelihoodTimesPrior=np.multiply(TMatrix[len(T)-1,:],np.transpose(TransitionMat[:,letter2idx[text[firstKnownLetterIndex]]]))
						Evidence=np.matmul(TransitionMat,TMatrix)[len(T)-1,letter2idx[text[firstKnownLetterIndex]]]
						Posterior=LikelihoodTimesPrior/Evidence
						cumulativeDist=np.cumsum(sorted(Posterior))
						randomNumber=np.random.random()
						for _ in range(len(T)):
							if cumulativeDist[_]>randomNumber:
								selectedLetterIndex=_
								break
						foundLetters[i]=(list(letter2idx.keys())[selectedLetterIndex])
						#knownLetterIndeces.append(i)
					else:
						if (i-1) in knownLetterIndeces:
							nextKnownLetterIndex=[nextIndex for nextIndex in knownLetterIndeces if nextIndex>i]
							if len(nextKnownLetterIndex)>0:
								nextKnownLetterIndex=nextKnownLetterIndex[0]
								TransitionMat=np.identity(len(T))
								for _ in range(nextKnownLetterIndex-i):
									TransitionMat=np.matmul(TransitionMat,TMatrix)
								LikelihoodTimesPrior=np.multiply(TMatrix[letter2idx[text[(i-1)]],:],np.transpose(TransitionMat[:,letter2idx[text[nextKnownLetterIndex]]]))
								Evidence=np.matmul(TransitionMat,TMatrix)[letter2idx[text[(i-1)]],letter2idx[text[nextKnownLetterIndex]]]
								Posterior=LikelihoodTimesPrior/Evidence
								cumulativeDist=np.cumsum(sorted(Posterior))
								randomNumber=np.random.random()
								for _ in range(len(T)):
									if cumulativeDist[_]>randomNumber:
										selectedLetterIndex=_
										break
								foundLetters[i]=(list(letter2idx.keys())[selectedLetterIndex])
								#knownLetterIndeces.append(i)
							else:
								Posterior=TMatrix[letter2idx[text[(i-1)]],:]
								#print(Posterior)
								cumulativeDist=np.cumsum(sorted(Posterior))
								randomNumber=np.random.random()
								for _ in range(len(T)):
									if cumulativeDist[_]>randomNumber:
										selectedLetterIndex=_
										break
								foundLetters[i]=(list(letter2idx.keys())[selectedLetterIndex])
						else:
							nextKnownLetterIndex=[nextIndex for nextIndex in knownLetterIndeces if nextIndex>i]
							if len(nextKnownLetterIndex)>0:
								nextKnownLetterIndex=nextKnownLetterIndex[0]
								TransitionMat=np.identity(len(T))
								for _ in range(nextKnownLetterIndex-i):
									TransitionMat=np.matmul(TransitionMat,TMatrix)
								LikelihoodTimesPrior=np.multiply(TMatrix[letter2idx[foundLetters[(i-1)]],:],np.transpose(TransitionMat[:,letter2idx[text[nextKnownLetterIndex]]]))
								Evidence=np.matmul(TransitionMat,TMatrix)[letter2idx[foundLetters[(i-1)]],letter2idx[text[nextKnownLetterIndex]]]
								Posterior=LikelihoodTimesPrior/Evidence
								#cumulativeDist=np.cumsum(sorted(TMatrix[letter2idx[foundLetters[i-1]],:]))
								cumulativeDist=np.cumsum(sorted(Posterior))
								randomNumber=np.random.random()
								for _ in range(len(T)):
									if cumulativeDist[_]>randomNumber:
										selectedLetterIndex=_
										break
								foundLetters[i]=(list(letter2idx.keys())[selectedLetterIndex])
								#knownLetterIndeces.append(i)
							else:
								Posterior=TMatrix[letter2idx[foundLetters[(i-1)]],:]
								cumulativeDist=np.cumsum(sorted(Posterior))
								randomNumber=np.random.random()
								for _ in range(len(T)):
									if cumulativeDist[_]>randomNumber:
										selectedLetterIndex=_
										break
								foundLetters[i]=(list(letter2idx.keys())[selectedLetterIndex])							
			currentText=copy.copy(text)
			currentText=list(currentText)
			for _ in unknownLetterIndeces:
				currentText[_]=foundLetters[_]
			print(''.join(currentText))
		print('')

def TextPredictor(test_strings):
	for text in test_strings:
		text='.'+text
		knownLetterIndeces=[i for i,letter in enumerate(text) if letter!='_']
		unknownLetterIndeces=[i for i,letter in enumerate(text) if letter=='_']
		foundLetters={}
		iterable=iter(enumerate(text))
		maxProb=1
		warning=False
		for __ in iterable:
			i=__[0]
			letter=__[1]
			if i in unknownLetterIndeces:
				previousKnownLetterIndex=[index for index in knownLetterIndeces if index<i][-1]
				nextKnownLetterIndex=[index for index in knownLetterIndeces if index>i]
				if nextKnownLetterIndex:
					nextKnownLetterIndex=nextKnownLetterIndex[0]
					unknownSequenceLength=nextKnownLetterIndex-previousKnownLetterIndex-1
					if unknownSequenceLength>3 and not warning:
						print('this might take a while..')
						warning=True
					jointProbabilityDistribution=np.zeros(tuple(unknownSequenceLength*[len(T)]))+1
					possibleCombinations=list(itertools.product(range(len(T)),repeat=unknownSequenceLength))
					TransitionMat=TMatrix
					for _ in range(unknownSequenceLength):
						TransitionMat=np.matmul(TransitionMat,TMatrix)
					denominator=TransitionMat[letter2idx[text[previousKnownLetterIndex]],letter2idx[text[nextKnownLetterIndex]]]
					for comb in possibleCombinations:
						jointProbabilityDistribution[comb]*=TMatrix[letter2idx[text[previousKnownLetterIndex]],comb[0]]
						for _ in range(unknownSequenceLength-1):
							jointProbabilityDistribution[comb]*=TMatrix[comb[_],comb[_+1]]
						jointProbabilityDistribution[comb]*=TMatrix[comb[-1],letter2idx[text[nextKnownLetterIndex]]]
				else:
					unknownSequenceLength=i-previousKnownLetterIndex
					if unknownSequenceLength>3 and not warning:
						print('this might take a while..')
						warning=True
					jointProbabilityDistribution=np.zeros(tuple(unknownSequenceLength*[len(T)]))+1
					possibleCombinations=list(itertools.product(range(len(T)),repeat=unknownSequenceLength))
					denominator=1
					for comb in possibleCombinations:
						jointProbabilityDistribution[comb]*=TMatrix[letter2idx[text[previousKnownLetterIndex]],comb[0]]
						for _ in range(unknownSequenceLength-1):
							jointProbabilityDistribution[comb]*=TMatrix[comb[_],comb[_+1]]
				maximumIndex=np.unravel_index(np.argmax(jointProbabilityDistribution, axis=None), jointProbabilityDistribution.shape)
				for _ in range(unknownSequenceLength):	
					foundLetters[i+_]=list(letter2idx.keys())[maximumIndex[_]]
				numerator=jointProbabilityDistribution[maximumIndex]
				for _ in range(unknownSequenceLength-1):
					temp=next(iterable,None)
				maxProb*=(numerator/denominator)
		currentText=copy.copy(text)
		currentText=list(currentText)
		for _ in unknownLetterIndeces:
			currentText[_]=foundLetters[_]
		del currentText[0]
		print('Most probable word:',''.join(currentText),'log(P(unknown|known))=',round(np.log(maxProb),4))
	print('')

10 samples are drawn for each test string

In [9]:
TextSampler(test_strings,10)
TextPredictor(test_strings)

therbroun.fex.
the.brhon.fex.
the.brzan.fex.
thambrd.n.fix.
the.br.in.fex.
tho.brven.fex.
the.br.in.fex.
theabroon.fex.
the.brman.fax.
thiobrean.fex.

munsthand.toube.inswere.
butst.ons.tombe.answeran
qugsthink.toube.answaren
cu.staknd.to.be.inswerth
ouastheny.to.be.inswer.b
bu.string.to.be.inswerer
bupsthind.to.be.inswerth
buasthind.toube.unsweran
ouastheng.to.be.inswormu
buestrenm.toube.onsweric

ithath..aghind.me.rding
is.aty.ba.hend.rearaing
ismat..wa.hond.he.r.ing
incath.pashend.vearaing
id.ate.wathend.betr.ing
inkate.wathong.herruing
iewatt.eaphen..wepreing
it.ath.mashand.he.r.ing
in.ati.pathend.befr.iag
iovate.mathang.jetrhing

qur.t..wiz.twid.t.wr.hai.me.
quprt.burz.omy.at.t..ice.ct.
qupet.aurz.d.f.st.ve.nsu.be.
qum.t.wirz.fe.act.se.n.t.be.
qunat.harz.in.ntt.wa.wat.tu.
qut.t.thiz.tfly.t.on.wly.go.
qus.t.t.az.ofouct.pe.she.be.
qurut.in.z.oo.s.t..g.f.h.ce.
qun.t.torz.thicat.ce.orn.w..
qud.t.runz...a..t.es.d.s.it.

Most probable word: the.br.an.fex. log(P(unknown|known))= -3.0743


The model could be improved by using a higher order Markov Chain, meaning in a $N^{th}$ order Markov Chain the probability of $ x_{t}$ is dependent on $ x_{t-1:t-N}$. The improved model would predict syllables instead of letters.